In [ ]:
%matplotlib inline
from osgeo import gdal, ogr, osr
import os, sys
import glob
import simplekml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import pandas as pd

sys.path.insert(0, os.path.dirname(os.getcwd()))
sys.path.insert(0, os.getcwd())


from utils.plots import plot_heatmap
import utils.gdal_processing as gp
from utils.read_geoTiff import readHR

p2ha = lambda x: (x*10)**2 /100**2

In [ ]:
def zonal_stats_old(FID, input_zone_polygon, input_value_raster, fn, is_return_numpoints = False, refband=1):

    # Open data
    raster = gdal.Open(input_value_raster)
    shp = ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()

    # Get raster georeference info
    transform = raster.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]

    # Reproject vector geometry to same projection as raster
    sourceSR = lyr.GetSpatialRef()
    targetSR = osr.SpatialReference()
    targetSR.ImportFromWkt(raster.GetProjectionRef())
    coordTrans = osr.CoordinateTransformation(sourceSR,targetSR)
    feat = lyr.GetFeature(FID)
    geom = feat.GetGeometryRef()
    geom.Transform(coordTrans)

    # Get extent of feat
    geom = feat.GetGeometryRef()

    if geom.GetGeometryName() == 'MULTIPOLYGON' :
        count = 0
        pointsX = []; pointsY = []
        for polygon in geom:
            geomInner = geom.GetGeometryRef(count)
            ring = geomInner.GetGeometryRef(0)
            numpoints = ring.GetPointCount()
            for p in range(numpoints):
                    lon, lat, z = ring.GetPoint(p)
                    pointsX.append(lon)
                    pointsY.append(lat)
            count += 1
    elif geom.GetGeometryName() == 'POLYGON':
        ring = geom.GetGeometryRef(0)
        numpoints = ring.GetPointCount()
        pointsX = []; pointsY = []
        for p in range(numpoints):
                lon, lat, z = ring.GetPoint(p)
                pointsX.append(lon)
                pointsY.append(lat)
    elif (geom.GetGeometryName() == 'LINESTRING'):
        numpoints = geom.GetPointCount()
        pointsX = []
        pointsY = []
        for p in range(numpoints):
            lon, lat, z = geom.GetPoint(p)
            pointsX.append(lon)
            pointsY.append(lat)
    else:
        sys.exit("ERROR: Geometry needs to be either Polygon or Multipolygon")

    xmin = min(pointsX)
    xmax = max(pointsX)
    ymin = min(pointsY)
    ymax = max(pointsY)

    # Specify offset and rows and columns to read
    xoff = int((xmin - xOrigin)/pixelWidth)
    yoff = int((yOrigin - ymax)/pixelWidth)
    if xoff < 0 or yoff < 0:
        return np.nan
    xcount = int((xmax - xmin)/pixelWidth)+1
    ycount = int((ymax - ymin)/pixelWidth)+1

    if is_return_numpoints:
        # TODO check that all the points are inside the region of interest
        return geom.GetPointCount()

    # Create memory target raster
    target_ds = gdal.GetDriverByName('MEM').Create('', xcount, ycount, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform((
        xmin, pixelWidth, 0,
        ymax, 0, pixelHeight,
    ))

    # Create for target raster the same projection as for the value raster
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(raster.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())

    # Rasterize zone polygon to raster
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values=[1])

    # Read raster as arrays
    banddataraster = raster.GetRasterBand(refband)
    try:
        dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(np.float)
    except AttributeError:
        return np.nan
    bandmask = target_ds.GetRasterBand(1)
    datamask = bandmask.ReadAsArray(0, 0, xcount, ycount).astype(np.float)
    print(datamask.mean())
    clip = True
    if clip:
        dataraster = np.clip(dataraster,0.01,1e9)
    if not np.any(datamask):
        print('datamask empty')
        return np.nan
    # Mask zone of raster
#     zoneraster = np.ma.masked_array(dataraster,  np.logical_not(datamask))
    dataraster[np.logical_not(datamask)] = np.nan

    # Calculate statistics of zonal raster
    # return numpy.average(zoneraster),numpy.mean(zoneraster),numpy.median(zoneraster),numpy.std(zoneraster),numpy.var(zoneraster)
    try:
        return fn(dataraster)
    except ValueError:
        print('fix')
        return np.nan
    
def loop_zonal_stats_update_old(input_zone_polygon, input_value_raster, fieldname, fn, is_update=True, refband=1, is_pos_only=False):

    shp = ogr.Open(input_zone_polygon, update=1)
    lyr = shp.GetLayer()
    lyrdf =lyr.GetLayerDefn()

    # TreeFieldName = 'TreePredAd1'
    if is_update:
        id_ = lyrdf.GetFieldIndex(fieldname)
        if id_ == -1:
            field_defn = ogr.FieldDefn(fieldname, ogr.OFTReal)
            lyr.CreateField(field_defn)
            id_ = lyrdf.GetFieldIndex(fieldname)
        else:
            print('Field {} already exists, may overwrite'.format(fieldname))
    outVals = []
    id_Name = lyrdf.GetFieldIndex('Name')
    for FID in range(lyr.GetFeatureCount()):
        feat = lyr.GetFeature(FID)
        if feat is not None:
            # compute sum
            name_ = feat.GetField(id_Name)
            if 'pos' in name_ or not is_pos_only:
                meanValue = zonal_stats(FID, input_zone_polygon, input_value_raster, fn, refband=refband)
                print(f' {meanValue:.2f} Trees in {name_}')

            else:
                meanValue = zonal_stats(FID, input_zone_polygon, input_value_raster, fn, is_return_numpoints=True, refband=refband)
                print(f' {meanValue:.2f} Ref points in {name_}')
            outVals.append(meanValue)
            if np.isnan(meanValue):
                print(meanValue,FID)
            if is_update:
                lyr.SetFeature(feat)
                feat.SetField(id_,meanValue)
                lyr.SetFeature(feat)
    return np.sum(outVals)


In [ ]:
def loop_zonal_stats_update(input_zone_polygon, input_value_raster, fieldname, fn, is_update=True, refband=1, is_pos_only=False,bias=1, field_name = 'Name'):

    shp = ogr.Open(input_zone_polygon, update=1)
    lyr = shp.GetLayer()
    lyrdf =lyr.GetLayerDefn()

    
    id_ = lyrdf.GetFieldIndex(fieldname)
    if id_ == -1:
        field_defn = ogr.FieldDefn(fieldname, ogr.OFTReal)
        lyr.CreateField(field_defn)
        id_ = lyrdf.GetFieldIndex(fieldname)
    else:
        print('Field {} already exists, may overwrite'.format(fieldname))
    outVals = []
    id_Name = lyrdf.GetFieldIndex(field_name)
    for FID in range(lyr.GetFeatureCount()):
        feat = lyr.GetFeature(FID)
        if feat is not None:
            # compute sum
            name_ = feat.GetField(id_Name)
            meanValue = zonal_stats(FID, input_zone_polygon, input_value_raster, fn, refband=refband,bias=bias)
#             print(f' {meanValue:.2f} Trees in {name_}')
            outVals.append(meanValue)
#             if np.isnan(meanValue):
#                 print(name_,FID,'is all nan')
            lyr.SetFeature(feat)
            feat.SetField(id_,meanValue)
            lyr.SetFeature(feat)
    return np.sum(outVals)

def zonal_stats(FID, input_zone_polygon, input_value_raster, fn, is_return_numpoints = False, refband=1, bias = 1.0):

    # Open data
    raster = gdal.Open(input_value_raster)
    shp = ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()

    # Get raster georeference info
    transform = raster.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]

    # Reproject vector geometry to same projection as raster
    sourceSR = lyr.GetSpatialRef()
    targetSR = osr.SpatialReference()
    targetSR.ImportFromWkt(raster.GetProjectionRef())
    coordTrans = osr.CoordinateTransformation(sourceSR,targetSR)
    feat = lyr.GetFeature(FID)
    geom = feat.GetGeometryRef()
    geom.Transform(coordTrans)

    # Get extent of feat
    geom = feat.GetGeometryRef()
    if (geom.GetGeometryName() == 'MULTIPOLYGON'):
        count = 0
        pointsX = []; pointsY = []
        for polygon in geom:
            geomInner = geom.GetGeometryRef(count)
            ring = geomInner.GetGeometryRef(0)
            numpoints = ring.GetPointCount()
            for p in range(numpoints):
                    lon, lat, z = ring.GetPoint(p)
                    pointsX.append(lon)
                    pointsY.append(lat)
            count += 1
    elif geom.GetGeometryName() == 'POLYGON':
        ring = geom.GetGeometryRef(0)
        numpoints = ring.GetPointCount()
        pointsX = []; pointsY = []
        for p in range(numpoints):
                lon, lat, z = ring.GetPoint(p)
                pointsX.append(lon)
                pointsY.append(lat)
    else:
        sys.exit("ERROR: Geometry needs to be a Polygon")
    xmin = min(pointsX)
    xmax = max(pointsX)
    ymin = min(pointsY)
    ymax = max(pointsY)

    # Specify offset and rows and columns to read
    xoff = int((xmin - xOrigin)/pixelWidth)
    yoff = int((yOrigin - ymax)/pixelWidth)
   
    xcount = int((xmax - xmin)/pixelWidth)+1
    ycount = int((ymax - ymin)/pixelWidth)+1


    xoff = min(xoff,raster.RasterXSize -1)
    xoff = max(xoff,1)
    
    xcount = min(xcount,raster.RasterXSize -1 - xoff)
    ycount = min(ycount,raster.RasterYSize -1 - yoff)
      

    # Create memory target raster
    target_ds = gdal.GetDriverByName('MEM').Create('', xcount, ycount, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform((
        xmin, pixelWidth, 0,
        ymax, 0, pixelHeight,
    ))

    # Create for target raster the same projection as for the value raster
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(raster.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())

    # Rasterize zone polygon to raster
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values=[1])

    # Read raster as arrays
    banddataraster = raster.GetRasterBand(refband)
    try:
        dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(np.float)
    except AttributeError:
        print('dataraster wrong')
#         print('geotransform',transform)
        print(xoff,yoff,xcount,ycount)
        print(raster.RasterXSize,raster.RasterYSize, 'xmax,ymax:',xoff+xcount,yoff+xcount)
        return np.nan
    
    bandmask = target_ds.GetRasterBand(1)
    datamask = bandmask.ReadAsArray(0, 0, xcount, ycount).astype(np.float)
#     print(datamask.mean())
    clip = True
    if clip:
#         dataraster = np.clip(dataraster,0.01,1e9)
        dataraster[dataraster < 0.01] = np.nan
    dataraster[dataraster == 99] = np.nan
    
    if not np.any(datamask):
        print('datamask empty')
        return np.nan
    # Mask zone of raster
#     zoneraster = np.ma.masked_array(dataraster,  np.logical_not(datamask))
    dataraster[np.logical_not(datamask)] = np.nan
    dataraster *=bias
    # Calculate statistics of zonal raster
    # return numpy.average(zoneraster),numpy.mean(zoneraster),numpy.median(zoneraster),numpy.std(zoneraster),numpy.var(zoneraster)
    try:
        return fn(dataraster)
    except ValueError:
        print('fix')
        return np.nan
    

In [ ]:
# obj='palm'

object_dict= {'palm':0,'coco':1}

# ref_band = object_dict[obj]

# points ='/home/pf/pfstud/andresro/tree_annotationsAug2019/annotations/Jan/palm/49MCV/Palm_Jan_1.kml'


In [ ]:
# TODO rasterize counts from annotations_folder to be able to compare them with the prediction including pos-neg areas

In [ ]:
# ref_folder = '/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations/T49NGD/group1' 
# ref_raster = '/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA9/T49NGD_preds_reg.tif'

# dens_min, dens_max = 0,3

# ds = gdal.Open(ref_raster)
# roi_ = gp.get_positive_area_folder(ref_folder)

# lims = gp.to_xy_box(roi_, ds, enlarge=10)

# raster = gp.rasterize_points_pos_neg_folder(folder=ref_folder,refDataset=ref_raster,lims=lims,lims_with_labels=lims,sq_kernel=2)

In [ ]:
# # raster.shape
# raster[raster == -1] = np.nan
# # plt.imshow(raster)
# p2ha = lambda x: (x*10)**2 /100**2

# scale = 50
# print(p2ha(scale))
# r1 = gp.block_reduce(raster,(scale,scale),np.nansum)
# # plt.imshow(r1)
# plt.imshow(r1-1,cmap = 'bwr',vmin=-1,vmax=1)


In [ ]:

# tile = 'T47NQA'
# sq_kernel=2
# folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5'
# ref_folder='/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/palm/kml_geoproposals'
# ref_raster = f'{folder_inference}/{tile}_preds_reg.tif'
# assert os.path.isfile(ref_raster), f'file {ref_raster} not found'
#         #     ref_folder = '/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations/T49NGD/group1' 
#         #     ref_raster = '/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA9/T49NGD_preds_reg.tif'
# dens_min, dens_max = 0,3

# ds = gdal.Open(ref_raster)
# #roi_ = '101.45,0.53,101.62,0.55'
# #roi_ = gp.get_positive_area_folder(ref_folder)
# lims = gp.to_xy_box(data_config[tile], ds, enlarge=10)
# print(lims)

# raster = gp.rasterize_points_constrained(Input=ref_folder,refDataset=ref_raster,lims=lims,lims_with_labels=lims,sq_kernel=2)

In [ ]:
# imshow(raster)

In [ ]:
# for automatic GT
data_config = {'T47NQA':'101.45,0.53,101.62,0.55'} 


In [ ]:
p2ha = lambda x: (x/10)**2
def plot_preds(scale,raster,preds, tile, group,density = (0,3)):
    dens_min,dens_max = density
    mask_out = np.isnan(raster) | np.isnan(preds)
#         gt_count = raster[~mask_out].sum()
    preds1 = preds.copy()
    preds1[mask_out] = np.nan
    raster1 = raster.copy()
    raster1[mask_out] = np.nan

    print(p2ha(scale))
    r1 = gp.block_reduce(raster1,(scale,scale),np.nansum)
    p1 = gp.block_reduce(preds1,(scale,scale),np.nansum)
    diff = p1  - r1
    diff1 = (p1 - r1 )/ r1
    diff1[np.logical_and(r1 == 0, p1 == 0) ] = 0
    diff1[np.logical_and(r1 == 0, p1 > 0) ] = np.nan

    fig = plt.figure(figsize=(10,5))
    n_col = 3
    gs = gridspec.GridSpec(nrows=1,ncols=n_col,left=0.02, bottom=0.06, right=0.95, top=0.94, wspace=0.1)

    txt = f' {tile} {group} {p2ha(scale)}ha'
    
    # GT
    ax = plt.subplot(gs[0])
    im = ax.imshow(raster,vmin=dens_min,vmax=dens_max)
    
    cbar = fig.colorbar(im,ax=ax,orientation='horizontal')
    cbar.ax.set_xlabel(f'Trees/ {p2ha(1):.2f}ha blocks')
    ax.set_title(f'Trees GT {np.nansum(raster1):.2f} ({np.nanmax(raster):.2f})')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(f' {preds1.shape[0]/100:.2f}km')
    ax.set_ylabel(f' {preds1.shape[1]/100:.2f}km')

    # PREDS
    ax = plt.subplot(gs[1])
    im = ax.imshow(preds,vmin=dens_min,vmax=dens_max)
    cbar = fig.colorbar(im,ax=ax,orientation='horizontal')
    cbar.ax.set_xlabel(f'Trees/ {p2ha(1):.2f}ha blocks')
    ax.set_title(f'Trees Pred {np.nansum(preds1):.2f} ({np.nanmax(preds):.2f})')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(f' {preds1.shape[0]/100:.2f}km \n\n'+txt)
    ax.set_ylabel(f' {preds1.shape[1]/100:.2f}km')

    # DIFSS
    ax = plt.subplot(gs[2])
    lim_ = dens_max * ((scale/2)**2)
    im = ax.imshow(diff,cmap = 'bwr', vmin = -lim_,vmax=lim_ ) #,vmin=-dens_max*scale*3,vmax=dens_max*scale*3)
    cbar = fig.colorbar(im,ax=ax,orientation='horizontal')
    cbar.ax.set_xlabel(f'Trees/ {p2ha(scale)}ha blocks')
    
    
    
    ax.set_title(f'Error per {p2ha(scale)}ha blocks')
    ax.set_xticks([])
    ax.set_yticks([])

    
#     fig.suptitle(f' {tile} {group} {p2ha(scale)}ha', y=1.01)
#     fig.text(.5, .06, txt, ha='center')


    # Scatter
    zeros_ = np.logical_and(p1== 0,r1==0).ravel()
    ds = pd.DataFrame({'GT':r1.ravel(),'Pred':p1.ravel()})
    return ds
def plot_counts(folder_inference, tile, folder_annotations, group='group1', preds_axis=0, sq_kernel=2, scale=10):
    ds_out = pd.DataFrame(columns=['GT','Pred'])
    
    ref_folders = glob.glob(f'{folder_annotations}/{tile}/{group}')
    if not ref_folders:
        print(f'no folders in {folder_annotations}/{tile}/{group}')
        return ds_out

    is_aut_gt = False
    for ref_folder in ref_folders:
        group = ref_folder.split('/')[-1]
        print(ref_folder, group)
        ref_raster = glob.glob(f'{folder_inference}/{tile}*_preds_reg*.tif')
        if len(ref_raster) == 0:
            print(f' no files found in {folder_inference}/{tile}*_preds_reg*.tif skipping...')
        else:
            ref_raster = ref_raster[0]
            #sys.stdout = open(os.devnull, "w")

            ds = gdal.Open(ref_raster)
            if is_aut_gt:
                return None
                roi_ = data_config[tile]
            else:
                roi_ = gp.get_positive_area_folder(ref_folder)
            lims = gp.to_xy_box(roi_, ds, enlarge=10)

            if is_aut_gt:
                raster = gp.rasterize_points_constrained(Input=ref_folder,refDataset=ref_raster,lims=lims,lims_with_labels=lims,sq_kernel=sq_kernel)
            else:
                raster = gp.rasterize_points_pos_neg_folder(folder=ref_folder,refDataset=ref_raster,lims=lims,lims_with_labels=lims,sq_kernel=sq_kernel)
            raster[raster == -1] = np.nan

            preds = readHR(data_file=ref_raster,roi_lon_lat=roi_,scale=10, as_float=False, is_assert_blank=False)
            sys.stdout = sys.__stdout__
            if preds.shape[-1] == 2:
                preds = preds[...,preds_axis]
            preds[preds <0.1] = 0
            preds[preds==99] = np.nan
        #     preds = np.clip(preds*1.3,0,2.5)

            ref_raster_sem = ref_raster.replace('reg.tif','semA.tif')

            if os.path.isfile(ref_raster_sem):
                preds_sem = readHR(data_file=ref_raster.replace('reg.tif','semA.tif'),roi_lon_lat=roi_,scale=10, as_float=False, is_assert_blank=False)
            else:
                preds_sem = preds > 0.5

            ds = plot_preds(scale,raster,preds, tile, group)
            ds_out = ds_out.append(ds)

        return ds_out
        


## Phillipines

In [ ]:
#folder_inference = '/home/pf/pfstaff/projects/andresro/sparse/inference_leon/borneo_simpleA9_mc10'
folder_inference = '/scratch/andresro/leon_work/sparse/inference/cocopreactive_simpleA9_soft_mc5'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])

tilenames = 'T50PNQ,T50PPR,T50PQS,T51NXA,T51NXB,T51NXG,T51NXH,T51NYB,T51NYG,T51NZH,T51NZJ,T51PUQ,T51PUR,T51PUT,T51PVR,T51PWK,T51PWQ,T51PWR,T51PYK,T51QUA'.split(',')
## forest areas
tilenames += 'T51QVV,T50PTT,T51PXL'.split(',')


for tile in tilenames:
        
    ds_ = plot_counts(folder_inference=folder_inference,
                tile=tile,
                folder_annotations='/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/manual_annotations',
                group='coconut_group2*', scale=scale)
    if ds_ is not None:
        ds_out = ds_out.append(ds_,ignore_index=True)
    else:
        print('error in tile',tile)

In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

# ds['Pred'] = ds.Pred*1.2

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds",
#                   kind="hex",
                 ) #, clip=(dens_min,dens_max))

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%) \n',x=-0.1,y=0.5, fontsize = 12)


In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA20clean_all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFC',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NGD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NEC',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NED',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)



In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.4

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
ds1 = ds_out.copy()
sys.stdout = sys.__stdout__


ds1['GT_sem'] = ds1.GT > 0
ds1['Pred_sem'] = ds1.Pred > 0


TP = np.logical_and(ds1.GT_sem == True,ds1.Pred_sem == True).sum()
TN = np.logical_and(ds1.GT_sem == False,ds1.Pred_sem == False).sum()
FP = np.logical_and(ds1.GT_sem == False,ds1.Pred_sem == True).sum()
FN = np.logical_and(ds1.GT_sem == True,ds1.Pred_sem == False).sum()


f' TP {TP} TN{TN}  FP {FP} FN {FN} GT {ds1.GT_sem.sum()*p2ha(scale)} ha Pred {ds1.Pred_sem.sum()*p2ha(scale)} ha Acc {(TP + TN ) / (TP + TN + FP + FN):.2f} Prec {(TP) / (TP + FP):.2f} Rec {(TP) / (TP + FN):.2f}  Iou   {(TP) / (TP + FN + FP):.2f}'  


## Palmcoco Evaluation

In [ ]:
folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmcocotiles2_simpleA'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/coco_annotations',
            group='group2',preds_axis=1, scale=scale)
# ds_out_ = ds_out.append(ds_,ignore_index=True)
ds_coco = ds_
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group2',preds_axis=0, scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT)):.2f} Trees/ {p2ha(scale)} ha  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    


zeros_ = np.logical_and(ds_coco.GT== 0,ds_coco.Pred==0)
ds = ds_coco[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT)):.2f} Trees/ {p2ha(scale)} ha  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA20westkalim'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49MCV',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/coco_annotations',
#             group='group',preds_axis=1, scale=scale)
# ds_out_ = ds_out.append(ds_,ignore_index=True)
# ds_coco = ds_
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*',preds_axis=0, scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT)):.2f} Trees/ {p2ha(scale)} ha  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    


zeros_ = np.logical_and(ds_coco.GT== 0,ds_coco.Pred==0)
ds = ds_coco[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT)):.2f} Trees/ {p2ha(scale)} ha  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmcocotiles2_coco_palm_kalim_simpleA'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/coco_annotations',
            group='group2',preds_axis=1, scale=scale)
# ds_out_ = ds_out.append(ds_,ignore_index=True)
ds_coco = ds_
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*',preds_axis=0, scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f'PALM MAE {np.nanmean(np.abs(ds.Pred -ds.GT)):.2f} Trees/ {p2ha(scale)} ha  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    


zeros_ = np.logical_and(ds_coco.GT== 0,ds_coco.Pred==0)
ds = ds_coco[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f'COCO MAE {np.nanmean(np.abs(ds.Pred -ds.GT)):.2f} Trees/ {p2ha(scale)} ha  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA9fix'
scale = 20
ds_out = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5'
scale = 20
ds_out = plot_counts(folder_inference=folder_inference,
            tile='T49MCV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5'
scale = 20
ds_out = plot_counts(folder_inference=folder_inference,
            tile='T47NQA',
            folder_annotations='/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/palm/kml_geoproposals',
            group='group*', scale=scale)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

#ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

## Palm SARAWAK

In [ ]:
folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA9fix'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFC',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NGD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NEC',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NED',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)





In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.4

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA20clean_all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFC',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NGD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NEC',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NED',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)



In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.4

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
ds1 = ds_out.copy()
sys.stdout = sys.__stdout__


ds1['GT_sem'] = ds1.GT > 0
ds1['Pred_sem'] = ds1.Pred > 0


TP = np.logical_and(ds1.GT_sem == True,ds1.Pred_sem == True).sum()
TN = np.logical_and(ds1.GT_sem == False,ds1.Pred_sem == False).sum()
FP = np.logical_and(ds1.GT_sem == False,ds1.Pred_sem == True).sum()
FN = np.logical_and(ds1.GT_sem == True,ds1.Pred_sem == False).sum()


f' TP {TP} TN{TN}  FP {FP} FN {FN} GT {ds1.GT_sem.sum()*p2ha(scale)} ha Pred {ds1.Pred_sem.sum()*p2ha(scale)} ha Acc {(TP + TN ) / (TP + TN + FP + FN):.2f} Prec {(TP) / (TP + FP):.2f} Rec {(TP) / (TP + FN):.2f}  Iou   {(TP) / (TP + FN + FP):.2f}'  


In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA9clean'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFC',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NGD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NEC',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NED',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
# folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA9clean'
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA9ft'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFC',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NGD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NEC',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NED',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.5

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsarawak3_simpleA20clean_all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFC',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NFD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T49NGD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NEC',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T49NED',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.4

plt.figure(figsize=(10,10))
g = sns.jointplot(x='Pred',y='GT',data=ds) #, cmap="Reds") #, clip=(dens_min,dens_max))
txt = f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f' Total Trees GT:{np.nansum(ds.GT):.2f}  Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)'
plt.suptitle(txt,y = 1.05) #, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    
txt

## Palm Sabah

In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmsabah_simpleA9all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T50NNL',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T50NNM',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.1

plt.figure(figsize=(10,10))
g = sns.jointplot(x='Pred',y='GT',data=ds) #, cmap="Reds") #, clip=(dens_min,dens_max))
txt = f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f' Total Trees GT:{np.nansum(ds.GT):.2f}  Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)'
plt.suptitle(txt,y = 1.05) #, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    
txt

## Palm Peninsula

In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmpeninsula_simpleA9clean_all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NRE',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)
ds_ = plot_counts(folder_inference=folder_inference,
            tile='T48NTK',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T48NTJ',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

# NEW
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T47NQD',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T48NUH',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T48NTK',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.3

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmpeninsula2_simpleAall'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T47NRE',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)
# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T48NTJ',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)


# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T48NTK',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NRG',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NPF',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NRF',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)



In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.1

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
# folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmpeninsula2_simpleA'
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmpeninsula_simpleA9clean_all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])
ds_ = plot_counts(folder_inference=folder_inference,
           tile='T47NRE',
           folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
           group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T48NTJ',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T48NTK',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.3

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

## Peninsulanew (all)

In [ ]:
# folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmpeninsula2_simpleA'
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmpeninsulanew_simpleA9all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NQD',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


# ds_ = plot_counts(folder_inference=folder_inference,
#            tile='T48NTK',
#            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#            group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

# ds_ = plot_counts(folder_inference=folder_inference,
#            tile='T48NTJ',
#            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#            group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T48NUH',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.1

plt.figure(figsize=(10,10))
g = sns.jointplot(x='Pred',y='GT',data=ds) #, cmap="Reds") #, clip=(dens_min,dens_max))
txt = f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f' Total Trees GT:{np.nansum(ds.GT):.2f}  Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)'
plt.suptitle(txt,y = 1.05) #, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    
txt 

## Riau

In [ ]:
# folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmpeninsula2_simpleA'
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmriau_simpleA9all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NQA',
            folder_annotations='/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/palm/kml_geoproposals',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T47NQA',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)



# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T47MQV',
#             folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

# ds['Pred'] = ds.Pred*1.4

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
# folder_inference = '/scratch/andresro/leon_work/sparse/inference/palmpeninsula2_simpleA'
folder_inference = '/scratch/andresro/leon_igp/sparse/inference/palmriau_simpleA9all'
scale=20
ds_out = pd.DataFrame(columns=['GT','Pred'])


# ds_ = plot_counts(folder_inference=folder_inference,
#             tile='T47NQA',
#             folder_annotations='/home/pf/pfstaff/projects/andresro/barry_palm/data/labels/palm/kml_geoproposals',
#             group='group*', scale=scale)
# ds_out = ds_out.append(ds_,ignore_index=True)

ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NQA',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group1', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NQA',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group2', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47NQA',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group3', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)


ds_ = plot_counts(folder_inference=folder_inference,
            tile='T47MQV',
            folder_annotations='/scratch/andresro/leon_igp/barry_palm/data/labels/palm_annotations',
            group='group*', scale=scale)
ds_out = ds_out.append(ds_,ignore_index=True)

In [ ]:
zeros_ = np.logical_and(ds_out.GT== 0,ds_out.Pred==0)
ds = ds_out[~zeros_].copy()

ds['Pred'] = ds.Pred*1.4

g = sns.jointplot(x='Pred',y='GT',data=ds, cmap="Reds") #, clip=(dens_min,dens_max))
plt.title(f' MAE {np.nanmean(np.abs(ds.Pred -ds.GT))/p2ha(scale):.2f} Trees/ha in {p2ha(scale)}ha Blocks  \n ' \
          f'total trees GT:{np.nansum(ds.GT):.2f} Pred:{np.nansum(ds.Pred):.2f} ({100*(np.nansum(ds.Pred)-np.nansum(ds.GT))/np.nansum(ds.GT):.2f}%)', y=1.1, fontsize = 12)

lims = (0,np.nanmax(ds.GT))
g.ax_marg_x.set_xlim(lims)
g.ax_marg_y.set_ylim(lims)

# lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')    

In [ ]:
def convert_to_shp(points):
    if points.endswith('.kml'):
        new_points=points.replace('.kml','.shp')
        srcDS = gdal.OpenEx(points)
        ds = gdal.VectorTranslate(new_points, srcDS, format='ESRI Shapefile')
        ds = None
        points = new_points
    return points 


def drop_all_butName(points):
    dataSource = ogr.Open(points, 1) 

    layer = dataSource.GetLayer()

    lyrdf = layer.GetLayerDefn()

    id_Name = lyrdf.GetFieldIndex('Name')
    attr_N = lyrdf.GetFieldCount()
    print(attr_N, id_Name)
    for i in range(attr_N):
        if not i == id_Name:
            layer.DeleteField(i)   
    attr_N = lyrdf.GetFieldCount()
    print(attr_N)
    dataSource = None


In [ ]:
ref_raster = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5/R132_T49MCV_preds_reg.tif'
for i in range(3):
    points = f'/home/pf/pfstud/andresro/tree_annotationsAug2019/annotations/Jan/palm/49MCV/Palm_Jan_{i+1}.kml'
    points = convert_to_shp(points)
    print(points)
    loop_zonal_stats_update(input_zone_polygon=points,input_value_raster=ref_raster,fieldname='pred1',fn=np.ma.sum, is_update=False, is_pos_only=True)

In [ ]:
ref_raster = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5/R018_T47NQA_preds_reg.tif'
points = '/home/pf/pfstud/andresro/tree_annotationsAug2019/annotations/Andres/palm/points_manual_2019.kml'
points = convert_to_shp(points)
print(points)
if os.path.isdir(points):
    pointsList = glob.glob(points+"/*.shp")
    print(f'processing {len(pointsList)} layers')
else:
    pointsList = [points]

for points in pointsList:
    
    out_val = loop_zonal_stats_update(input_zone_polygon=points,input_value_raster=ref_raster,fieldname='pred1',fn=np.ma.sum, is_update=False)
    print(f"TOTAL {out_val:.2f}", os.path.basename(points))

## Evaluate state-wide predictions

In [ ]:
scale = 2
input_zone_pol = '/scratch2/Dropbox/Dropbox/0_phd/tree_annotationsAug2019/countries/malaysia/MYS_adm1.shp'
ref_raster = f'/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA20_allsarawak/0_untiled_down{scale}.tif'

In [ ]:

img = gp.rasterize_polygons(InputVector=input_zone_pol,refDataset=ref_raster,attribute='ID_1')

raster = readHR(roi_lon_lat = None,data_file=ref_raster,scale=1,as_float=False)

In [ ]:
# plt.imshow(img==11)
raster[img!=11] = np.nan
raster *=(scale**2) # computing sum instead of average in down op 
raster*=1.4 # bias correction

plt.imshow(raster)
plt.colorbar()

In [ ]:
p2ha(scale)

In [ ]:
np.nanmin(raster),np.nanmax(raster)

#### Statistics

In [ ]:
# Total Ha
ref_ = raster  > 30*p2ha(scale)
plt.imshow(ref_)
area = np.nansum(ref_)*p2ha(scale)
trees = np.nansum(raster[ref_])
f' Planted {area/1e6:.3f}mha , Trees {trees/1e6:.3f}m, Tree/Ha {trees/area} '

In [ ]:


out_val = loop_zonal_stats_update(input_zone_polygon=input_zone_pol,input_value_raster=ref_raster,fieldname='pred1',fn=np.nansum, is_update=False,bias=14.0, field_name='NAME_1')



In [ ]:
shp = ogr.Open(input_zone_pol, update = 1)

lyr = shp.GetLayer()

lyr

### Split raster into blocks and save it as shapefile

In [ ]:
import math
EARTH_RADIUS = 6371000  # Radius in meters of Earth


# Compute the shortest path curved distance between 2 points (lat1,lon1) and (lat2,lon2) using the Haversine formula.
def haversine_distance(lon1, lat1, lon2, lat2):

    a = math.sin(math.radians((lat2 - lat1) / 2.0)) ** 2 + math.cos(math.radians(lat1)) * math.cos(
        math.radians(lat2)) * math.sin(math.radians((lon2 - lon1) / 2.0)) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return EARTH_RADIUS * c

def split_roi_to_rois(lon1_, lat1_, lon2_, lat2_, meters_split = 1500):

    lon1, lat1, lon2, lat2 = min(lon1_, lon2_), min(lat1_,lat2_), max(lon1_, lon2_), max(lat1_, lat2_)

    delta_lon_m = haversine_distance(lon1=lon1,lat1=lat1,lon2=lon2,lat2=lat1)
    delta_lat_m = haversine_distance(lon1=lon1,lat1=lat1,lon2=lon1,lat2=lat2)
    rois = []

    N_lon, N_lat = map(lambda x: int(math.ceil(x / meters_split)), [delta_lon_m,delta_lat_m])

    delta_lon, delta_lat = (lon2-lon1, lat2 - lat1)
    for i in range(N_lat):
        for j in range(N_lon):
            ind = i * N_lon + j
            rois.append({"roi": (
                                lat1 + (delta_lat) * i / N_lat,
                                lon1 + (delta_lon) * j / N_lon,
                                lat1 + (delta_lat) * (i + 1) / N_lat,
                                lon1 + (delta_lon) * (j + 1) / N_lon),
                        "name": "ROI{}".format(ind + 1)})

    return rois

def to_bbox(roi_lon_lat):
    if isinstance(roi_lon_lat, str):
        roi_lon1, roi_lat1, roi_lon2, roi_lat2 = map(float, re.split(',', roi_lon_lat))
    else:
        roi_lon1, roi_lat1, roi_lon2, roi_lat2 = roi_lon_lat

    geo_pts_ref = [(roi_lon1, roi_lat1), (roi_lon1, roi_lat2), (roi_lon2, roi_lat2), (roi_lon2, roi_lat1)]
    return geo_pts_ref



def convert_to_shp(points, is_overwrite=False):
    if points.endswith('.kml'):
        new_points=points.replace('.kml','.shp')
        if not os.path.exists(new_points) or is_overwrite:
            srcDS = gdal.OpenEx(points)
            ds = gdal.VectorTranslate(new_points, srcDS, format='ESRI Shapefile')
            ds = None
            points = new_points
    return points 


In [ ]:
ref_raster = '/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA20/T49MCV_preds_reg.tif'
# ref_raster = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5/T49MCV_preds_reg.tif'
# save_dir = '/scratch/andresro/leon_work/sparse/inference/palmcoco_kalimA_simpleA5'
save_dir = '/scratch/andresro/leon_work/sparse/inference/palmsarawak_simpleA20/'
ds = gdal.Open(ref_raster)

geo_pts_ref = gp.get_lonlat(ds)
lon1_, lat1_, lon2_, lat2_ = min([x[0] for x in geo_pts_ref]), min([x[1] for x in geo_pts_ref]),max([x[0] for x in geo_pts_ref]), max([x[1] for x in geo_pts_ref])

roi_ = split_roi_to_rois(lon1_, lat1_, lon2_, lat2_,2000)
print(len(roi_))

In [ ]:
fname = os.path.basename(ref_raster).replace('.tif','')
kmlfile_name = f"{save_dir}/{fname}_rois_{len(roi_)}.kml"
kml = simplekml.Kml()
for roi in roi_:
    lat1, lon1, lat2, lon2 = roi["roi"]
    # print roi

    geo_pts_ref = to_bbox([lon1, lat1, lon2, lat2])
    pol = kml.newpolygon(name=roi['name'])
    pol.outerboundaryis = geo_pts_ref
    pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.white)

kml.save(kmlfile_name)
print(kmlfile_name)

In [ ]:
points = convert_to_shp(kmlfile_name, is_overwrite=True)
print(points)
# drop_all_butName(points)

In [ ]:
loop_zonal_stats_update(input_zone_polygon=points,input_value_raster=ref_raster,fieldname='pred_palm',fn=np.nansum, is_update=True, refband=1,bias=1.5)

In [ ]:
points

In [ ]:
kmlfile_name